*Теоретический материал:* https://youtu.be/sOTh6klME_M?t=139

*Разбор данного раздела:* https://youtu.be/sOTh6klME_M?t=3344

In [ ]:
from flask import Flask # Импортируем модуль Flask 

In [ ]:
app = Flask(__name__) # Создаем экземпляр Flask (аргумнтом передается имя модуля или пакета)

In [ ]:
@app.route('/') # Декоратор, указывает, что URL будет вызывать нашу функцию
def hello(): # Задаем функцию для вывода сообщения
    return 'Hello, WORLD!!!'

In [ ]:
import threading # Импортируем модуль threading для запуска в отдельном потоке

In [ ]:
threading.Thread(target = app.run, kwargs = {'host' : 'localhost', 'port':5000}).start() # Запускаем в отдельном потоке метод app.run, который запускает локальный сервер

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)


In [ ]:
import requests # Библиотека для HTTP-запросов

In [ ]:
r = requests.get('http://localhost:5000') # Запрашиваем данные с локального сервера по порту 5000
print(r.status_code) # Статус ответа
print(r.encoding) # Кодировка после декодирования
print(r.apparent_encoding) # Текущая кодировка
print(r.text) # Соержимое страницы

127.0.0.1 - - [10/Feb/2021 19:20:17] "GET / HTTP/1.1" 200 -


200
utf-8
ascii
Hello, WORLD!!!


# LIGHT

Немного изменил и дообучил модель, чтобы на тестовой выборке был лучший результат.
Исправления в предикт внес, теперь модель предсказывает как положено.
Внизу есть ссылки на скриншоты веб-страницы

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [ ]:
# Загружаем MNIST
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# Нормализуем значения пикспелей (от 0 до 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Приводим лейблы к формату One Hot
y_train = tensorflow.keras.utils.to_categorical(y_train, 10)
y_test = tensorflow.keras.utils.to_categorical(y_test, 10)

print(x_train.shape, y_train.shape)

(60000, 28, 28, 1) (60000, 10)


In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(2, 2),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_data=(x_test, y_test))

# Результат модели
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


# Сохраняем саму модель
# Сериализация в JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# Сохраняем(сериализуем) веса  в формате HDF5
model.save_weights("model.h5")
print("Сохранено")

Epoch 1/50
469/469 [==============================] - 2s 4ms/step - loss: 2.2122 - accuracy: 0.5005 - val_loss: 2.1792 - val_accuracy: 0.5685
Epoch 2/50
469/469 [==============================] - 2s 4ms/step - loss: 2.1406 - accuracy: 0.6117 - val_loss: 2.0908 - val_accuracy: 0.6303
Epoch 3/50
469/469 [==============================] - 2s 4ms/step - loss: 2.0328 - accuracy: 0.6506 - val_loss: 1.9573 - val_accuracy: 0.6621
Epoch 4/50
469/469 [==============================] - 2s 4ms/step - loss: 1.8731 - accuracy: 0.6761 - val_loss: 1.7647 - val_accuracy: 0.6849
Epoch 5/50
469/469 [==============================] - 2s 4ms/step - loss: 1.6553 - accuracy: 0.6957 - val_loss: 1.5181 - val_accuracy: 0.7143
Epoch 6/50
469/469 [==============================] - 2s 4ms/step - loss: 1.4003 - accuracy: 0.7244 - val_loss: 1.2553 - val_accuracy: 0.7479
Epoch 7/50
469/469 [==============================] - 2s 4ms/step - loss: 1.1552 - accuracy: 0.7557 - val_loss: 1.0282 - val_accuracy: 0.7784
Epoch 

In [ ]:
from flask import Flask, request # request - это объекты, которыми управляет flask handles (get, set, post и т.д.)
from flask import render_template # этот объект автоматически зарендерит нам шаблон html
import imageio # для работы с изображениями
import numpy as np
import tensorflow.keras.models
import re # регулярные питоновские выражения
from tensorflow.keras.models import model_from_json
import sys 
import os
from tensorflow.keras.preprocessing import image
# Задаем имя серверу
app = Flask(__name__)

import tensorflow as tf
import base64 # универсальный формат кодирования (часто используется с изображениями)


In [ ]:
# Открываем json файл разметки модели
json_file = open('model.json','r') 
loaded_model_json = json_file.read() # считываем
json_file.close() # закрываем
loaded_model = model_from_json(loaded_model_json) # используем керас, чтобы считать разметку архитектуры
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 27, 27, 64)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        8224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 702,186
Trainable params: 702,186
Non-trainable params: 0
________________________________________________

In [ ]:
loaded_model.load_weights("model.h5") # подгружаем веса
print("Загружено с диска")

Загружено с диска


In [ ]:
# компилируем модель
loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# оценим ее точность еще раз
loss,accuracy = loaded_model.evaluate(x_test,y_test)
print('accuracy:', accuracy)

313/313 [==============================] - 0s 1ms/step - loss: 0.2788 - accuracy: 0.9199
accuracy: 0.9199000000953674


In [ ]:
# декодируем изображение из сырого(бинарного) представления в его base64 
def convertImage(imgData1):
    imgstr = re.search(b'base64,(.*)',imgData1).group(1) # находим строчку, в которой содержится побитовое представление изображение,
                                                        # возвращает вторую(1) подгруппу
    with open('output.png','wb') as output: # временный файл, в который записываем декодированное представление
        output.write(base64.b64decode(imgstr))

In [ ]:
# # Указываем адрес по которому будет откликаться Web сервис и задаем саму функцию веб сервиса. Это наша стартовая страница
@app.route('/')
@app.route('/index')
def index():
    # рендерим заранее построенный HTML файл
    return render_template("index.html")

In [ ]:
# маршрут, по которому будет идти предсказание
@app.route('/predict/',methods=['GET','POST'])
def predict():
    # в любой момент, когда пользователь вызовет predict
    # после того, как нарисовал картинку, произойдет
    # inference/внедрение, вернется результат классификации
    
    # получить сырой формат изображения (этот метод объекта Flask)
    imgData = request.get_data()
    
    # кодируем в подходящий формат
    convertImage(imgData)
    print ("debug")
    # считываем изображение в память
    img = image.load_img('output.png', target_size=(28,28), color_mode = 'grayscale')
    sample = np.array(img)
    sample = np.reshape(sample, (1, 28, 28, 1))
    
    sample = sample.astype('float32')  
    sample /= 255
    sample = 1 - sample
    out = loaded_model.predict(sample)
    print ("debug2")               
    print(out)
    print(np.argmax(out,axis=1))
    print ("debug3")

# преобразуем ответ в строку
    response = np.array_str(np.argmax(out,axis=1))
    return response
        # обнуляем граф, чтобы можно было ввести новые данные
    #tf.reset_default_graph()

In [ ]:
#if __name__ == "__main__":
    # этот порт 
# выделяем под прогонку
port = int(os.environ.get('PORT', 5002))
    
    # прогоняем на локальном сервере по заданному порту
app.run(host='127.0.1.1', port=port, threaded=False)    
    # опционально, если хотите прогнать в режиме дебага
    #app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.1.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Feb/2021 22:15:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Feb/2021 22:15:15] "POST /predict/ HTTP/1.1" 200 -


debug
debug2
[[0.02164573 0.02487723 0.10147759 0.03190246 0.0240628  0.30421692
  0.09399462 0.01927641 0.24955131 0.12899497]]
[5]
debug3


127.0.0.1 - - [10/Feb/2021 22:15:19] "POST /predict/ HTTP/1.1" 200 -


debug
debug2
[[0.0008247  0.06062439 0.02387094 0.00821761 0.7145198  0.06292248
  0.0588288  0.00232022 0.03426072 0.03361034]]
[4]
debug3


...

debug
debug2
[[1.6022343e-02 7.6664874e-05 1.2624348e-02 7.4124819e-01 7.9039549e-03
  7.0492551e-02 3.4171392e-03 3.6993355e-03 1.3510782e-01 9.4076823e-03]]
[3]
debug3


127.0.0.1 - - [10/Feb/2021 22:16:40] "POST /predict/ HTTP/1.1" 200 -


debug
debug2
[[8.5208677e-03 1.0698794e-04 9.0810368e-03 8.4848768e-01 2.1452145e-03
  8.0007695e-02 6.0614566e-03 2.0122321e-03 4.3020140e-02 5.5671425e-04]]
[3]
debug3


127.0.0.1 - - [10/Feb/2021 22:16:44] "POST /predict/ HTTP/1.1" 200 -


debug
debug2
[[0.01550004 0.0027795  0.02322822 0.03460707 0.19417545 0.01133639
  0.53084326 0.00806868 0.1536566  0.02580478]]
[6]
debug3


# Скрины примеров работы

https://drive.google.com/file/d/1VgMs0t7HFiaP2PumCRsdViDcmWecTDU7/view?usp=sharing

https://drive.google.com/file/d/1r3wuthxJlqeVYdYF8lfMkV2SNCRCKsJn/view?usp=sharing